In [16]:
import pandas as pd
import requests
from datetime import datetime
from sqlalchemy import create_engine
import pickle
import os
uri = os.environ.get('URI')

In [17]:
# Load Files
forecaster_rf = pickle.load(open('model.pickle', 'rb'))
last_training_date = pickle.load(open('last_training_date.pickle', 'rb'))
last_training_date = datetime.strptime(last_training_date, '%Y-%m-%d %H:%M:%S')

In [18]:
forecaster_rf

ForecasterAutoreg 
Regressor: RandomForestRegressor(max_depth=10, random_state=123) 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20] 
Transformer for y: None 
Transformer for exog: None 
Window size: 20 
Weight function included: False 
Differentiation order: None 
Exogenous included: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: [Timestamp('2024-04-04 05:00:00'), Timestamp('2024-04-11 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: H 
Regressor parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 10, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0, 'warm_start': False} 
fit_kwargs: {} 
Creation date: 2024-04-12 14:26:33 
Last fit date: 2024-

In [19]:
url = 'https://api.blockchain.info/charts/transactions-per-second?timespan=all&sampled=false&metadata=false&cors=true&format=json'
resp = requests.get(url)
data = pd.DataFrame(resp.json()['values'])

# I correct the date
data['x'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in data['x']]
data['x'] = pd.to_datetime(data['x'])

# I read the last prediction
engine = create_engine(uri)
query = engine.execute('SELECT MAX(prediction_date) FROM predictions;')
last_prediction_date= query.fetchall()[0][0]
query.close()

In [20]:
print(f"LAST PREDICTION : {last_prediction_date}")
print(f"LAST TRAINING {last_training_date}")

LAST PREDICTION : 2024-04-08 00:00:00
LAST TRAINING 2024-04-12 12:00:00


In [21]:
# If there is no last date in the databse or training > database, I read the last date from training

num_predictions = 168

# if statements dostarczone przez neptune.ai chyba nie są poprawne, trzeba zmienić 
# na odpowiednie !!!

# poprawiony pierwszy if, po prostu sytaucja kiedy mamy NAJNOWSZY wytrenowany model, gotowy do użycia
if  (last_prediction_date is None) or (last_prediction_date < last_training_date):

    print("OPTION 1")

    # As there is no predictions, I make the predictions
    predictions = forecaster_rf.predict(num_predictions)

    fechas = pd.date_range(
        start = last_training_date.strftime('%Y-%m-%d %H:%M:%S'),
        periods = num_predictions,
        freq = '1H'
        )

# sytuacje gdzie już używaliśmy naszego wytrenowanego modelu to stworzenia jakiejś predykcji
elif last_prediction_date > last_training_date:

    print("OPTION 2")

    # In this case, we must take into account the differences between the last forecast date and add the difference to the number of days to extract.
    dif_seg= last_prediction_date - last_training_date
    hours_extract = num_predictions + dif_seg.seconds//3600

    # experiment
    num_predictions = dif_seg.seconds//3600
    predictions = forecaster_rf.predict(hours_extract)
    
    # I get the last predictions
    # wybieramy tylko NOWE predykcje
    predictions = predictions[-(num_predictions):]

    fechas = pd.date_range(
        start = last_prediction_date.strftime('%Y-%m-%d %H:%M:%S'),
        periods = num_predictions,
        freq = '1H'
        )
# is this ever used ? only in edge case scenario where some varaible is None or training_date == last_predict_date
else:

    print("OPTION 3")

    # If last training > last predictions
    predictions = forecaster_rf.predict(num_predictions)

    fechas = pd.date_range(
        start = last_training_date.strftime('%Y-%m-%d %H:%M:%S'),
        periods = num_predictions,
        freq = '1H'
        )
    
    

fechas

OPTION 1


DatetimeIndex(['2024-04-12 12:00:00', '2024-04-12 13:00:00',
               '2024-04-12 14:00:00', '2024-04-12 15:00:00',
               '2024-04-12 16:00:00', '2024-04-12 17:00:00',
               '2024-04-12 18:00:00', '2024-04-12 19:00:00',
               '2024-04-12 20:00:00', '2024-04-12 21:00:00',
               ...
               '2024-04-19 02:00:00', '2024-04-19 03:00:00',
               '2024-04-19 04:00:00', '2024-04-19 05:00:00',
               '2024-04-19 06:00:00', '2024-04-19 07:00:00',
               '2024-04-19 08:00:00', '2024-04-19 09:00:00',
               '2024-04-19 10:00:00', '2024-04-19 11:00:00'],
              dtype='datetime64[ns]', length=168, freq='H')

In [22]:
predictions

2024-04-11 01:00:00    17.757607
2024-04-11 02:00:00    25.452292
2024-04-11 03:00:00    22.885942
2024-04-11 04:00:00    27.657327
2024-04-11 05:00:00    21.016053
                         ...    
2024-04-17 20:00:00    35.882298
2024-04-17 21:00:00    35.882298
2024-04-17 22:00:00    35.882298
2024-04-17 23:00:00    35.882298
2024-04-18 00:00:00    35.882298
Freq: H, Name: pred, Length: 168, dtype: float64

In [25]:
upload_data = list(zip([
datetime.now().strftime('%Y-%m-%d %H:%M:%S')] * num_predictions,
[fecha.strftime('%Y-%m-%d %H:%M:%S') for fecha in fechas ],
    predictions
))


print(f"num of predictions {len(upload_data)}")

upload_data[:10]



num of predictions 168


[('2024-04-12 14:45:34', '2024-04-12 12:00:00', 17.757607142857143),
 ('2024-04-12 14:45:34', '2024-04-12 13:00:00', 25.452291917547786),
 ('2024-04-12 14:45:34', '2024-04-12 14:00:00', 22.88594174000204),
 ('2024-04-12 14:45:34', '2024-04-12 15:00:00', 27.65732663398693),
 ('2024-04-12 14:45:34', '2024-04-12 16:00:00', 21.016053401663697),
 ('2024-04-12 14:45:34', '2024-04-12 17:00:00', 27.44218697829949),
 ('2024-04-12 14:45:34', '2024-04-12 18:00:00', 21.32696475376477),
 ('2024-04-12 14:45:34', '2024-04-12 19:00:00', 35.25446212861211),
 ('2024-04-12 14:45:34', '2024-04-12 20:00:00', 24.010977819548863),
 ('2024-04-12 14:45:34', '2024-04-12 21:00:00', 33.27200537656786)]